# 📊 Análise de Gaps Temporais - Troca de Notas TERLOC

**Objetivo:** Medir gaps temporais no processo de troca de notas e identificar gargalos.

## Gaps Principais a Medir:
1. **Gap Cliente:** Retorno Simbólico → Recebimento NF Venda 
2. **Gap Pátio:** Ticket → Gate → Liberação
3. **Processos Incompletos:** Linhas sem todos os horários

## Processo TERLOC:
Terminal → Retorno Simbólico → Cliente emite NF Venda → TERLOC recebe → CEPARKING libera

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas carregadas com sucesso!")

✅ Bibliotecas carregadas com sucesso!


In [6]:
# Carregar planilha TERLOC - ABA CORRETA COM DADOS OPERACIONAIS
arquivo_excel = 'PLANILHA TROCA DE NOTA TERLOC.xlsx'

try:
    # Tentar carregar todas as abas
    excel_file = pd.ExcelFile(arquivo_excel)
    print(f"📋 Abas encontradas: {excel_file.sheet_names}")
    
    # ⚠️ CARREGAR ABA 'PLANILHA ÚNICA' - DADOS OPERACIONAIS COMPLETOS
    print(f"\n🎯 CARREGANDO ABA 'PLANILHA ÚNICA' COM DADOS OPERACIONAIS...")
    df = pd.read_excel(arquivo_excel, sheet_name='PLANILHA ÚNICA')
    print(f"✅ Dados operacionais carregados: {len(df):,} registros")
    print(f"🔍 Colunas disponíveis: {len(df.columns)} colunas")
    
    # Mostrar primeiras colunas importantes
    colunas_importantes = []
    for col in df.columns[:20]:  # Primeiras 20 colunas
        if any(palavra in str(col).lower() for palavra in ['data', 'hora', 'cliente', 'expedição', 'motorista']):
            colunas_importantes.append(col)
    
    print(f"🎯 Colunas importantes identificadas: {colunas_importantes}")
    
except Exception as e:
    print(f"❌ Erro ao carregar planilha: {e}")

📋 Abas encontradas: ['DADOS TERLOC', 'DADOS CEPARKING', 'Validação de dados', 'PLANILHA ÚNICA', 'Legenda']

🎯 CARREGANDO ABA 'PLANILHA ÚNICA' COM DADOS OPERACIONAIS...
✅ Dados operacionais carregados: 1,045,190 registros
🔍 Colunas disponíveis: 179 colunas
🎯 Colunas importantes identificadas: ['DATA', 'EXPEDIÇÃO', 'MOTORISTA', 'CLIENTE', 'CLIENTE DE VENDA', 'DATA  EMISSÃO RETORNO SIMBÓLICO', 'HORA  EMISSÃO RETORNO SIMBÓLICO', 'HORA RECEBIMENTO NF DE VENDA', 'DATA  TICKET', 'HORA TICKET', 'HORARIO SENHA ', 'HORA GATE ', 'DATA DE LIBERAÇÃO', 'HORARIO DE LIBERAÇÃO']
✅ Dados operacionais carregados: 1,045,190 registros
🔍 Colunas disponíveis: 179 colunas
🎯 Colunas importantes identificadas: ['DATA', 'EXPEDIÇÃO', 'MOTORISTA', 'CLIENTE', 'CLIENTE DE VENDA', 'DATA  EMISSÃO RETORNO SIMBÓLICO', 'HORA  EMISSÃO RETORNO SIMBÓLICO', 'HORA RECEBIMENTO NF DE VENDA', 'DATA  TICKET', 'HORA TICKET', 'HORARIO SENHA ', 'HORA GATE ', 'DATA DE LIBERAÇÃO', 'HORARIO DE LIBERAÇÃO']


In [5]:
# Verificar todas as abas da planilha para encontrar os dados corretos
print("🔍 ANALISANDO TODAS AS ABAS DA PLANILHA:")

for i, aba in enumerate(excel_file.sheet_names):
    print(f"\n📋 ABA {i+1}: '{aba}'")
    try:
        df_temp = pd.read_excel(arquivo_excel, sheet_name=aba)
        print(f"   📊 Linhas: {len(df_temp)}")
        print(f"   📋 Colunas: {len(df_temp.columns)}")
        print(f"   📝 Nomes das colunas: {list(df_temp.columns)[:10]}...")  # Primeiras 10 colunas
        
        # Verificar se tem colunas de tempo
        colunas_tempo = []
        for col in df_temp.columns:
            col_lower = str(col).lower()
            if any(palavra in col_lower for palavra in ['data', 'hora', 'ticket', 'gate', 'liberação', 'retorno']):
                colunas_tempo.append(col)
        
        if colunas_tempo:
            print(f"   ⏰ COLUNAS DE TEMPO ENCONTRADAS: {colunas_tempo}")
        else:
            print(f"   ❌ Nenhuma coluna de tempo encontrada")
            
    except Exception as e:
        print(f"   ❌ Erro ao ler aba: {e}")

print("\n" + "="*60)

🔍 ANALISANDO TODAS AS ABAS DA PLANILHA:

📋 ABA 1: 'DADOS TERLOC'
   📊 Linhas: 77
   📋 Colunas: 4
   📝 Nomes das colunas: ['TERLOC', 'E-MAIL', 'Unnamed: 2', 'Unnamed: 3']...
   ❌ Nenhuma coluna de tempo encontrada

📋 ABA 2: 'DADOS CEPARKING'
   📊 Linhas: 59
   📋 Colunas: 3
   📝 Nomes das colunas: ['CEPARKING', 'Unnamed: 1', 'Unnamed: 2']...
   ❌ Nenhuma coluna de tempo encontrada

📋 ABA 3: 'Validação de dados'
   📊 Linhas: 20
   📋 Colunas: 3
   📝 Nomes das colunas: ['CLIENTES VENDA', 'Unnamed: 1', 'CLIENTES DE ORIGEM']...
   ❌ Nenhuma coluna de tempo encontrada

📋 ABA 4: 'PLANILHA ÚNICA'
   📊 Linhas: 1045190
   📋 Colunas: 179
   📝 Nomes das colunas: ['DATA', 'EXPEDIÇÃO', 'MOTORISTA', 'PLACA', 'CLIENTE', 'CLIENTE DE VENDA', 'POSSUI NF DE CONTA E ORDEM?', 'DATA  EMISSÃO RETORNO SIMBÓLICO', 'HORA  EMISSÃO RETORNO SIMBÓLICO', 'RETORNO SIMBÓLICO ']...
   ⏰ COLUNAS DE TEMPO ENCONTRADAS: ['DATA', 'DATA  EMISSÃO RETORNO SIMBÓLICO', 'HORA  EMISSÃO RETORNO SIMBÓLICO', 'RETORNO SIMBÓLICO ', 'HORA 

In [ ]:
# Analisar estrutura dos dados operacionais REAIS
print("📋 ESTRUTURA DOS DADOS OPERACIONAIS:")
print(f"📊 Total de Registros: {len(df):,}")
print(f"📋 Total de Colunas: {len(df.columns)}")

# Verificar dados dos últimos meses (dados mais recentes)
print(f"\n🔍 ANALISANDO DADOS RECENTES:")
df_recente = df[df['DATA'].notna()].copy()
if len(df_recente) > 0:
    df_recente['DATA'] = pd.to_datetime(df_recente['DATA'], errors='coerce')
    df_recente = df_recente[df_recente['DATA'].notna()]
    
    if len(df_recente) > 0:
        data_min = df_recente['DATA'].min()
        data_max = df_recente['DATA'].max()
        print(f"📅 Período dos dados: {data_min.date()} até {data_max.date()}")
        
        # Filtrar últimos 30 dias para análise
        from datetime import datetime, timedelta
        data_corte = data_max - timedelta(days=30)
        df_ultimos_30 = df_recente[df_recente['DATA'] >= data_corte]
        print(f"📊 Registros últimos 30 dias: {len(df_ultimos_30):,}")

print(f"\n🎯 COLUNAS CRÍTICAS PARA ANÁLISE DE GAPS:")
colunas_gaps = ['DATA', 'EXPEDIÇÃO', 'MOTORISTA', 'CLIENTE', 'CLIENTE DE VENDA', 
                'DATA  EMISSÃO RETORNO SIMBÓLICO', 'HORA  EMISSÃO RETORNO SIMBÓLICO',
                'HORA RECEBIMENTO NF DE VENDA', 'DATA  TICKET', 'HORA TICKET',
                'HORARIO SENHA ', 'HORA GATE ', 'DATA DE LIBERAÇÃO', 'HORARIO DE LIBERAÇÃO']

for col in colunas_gaps:
    if col in df.columns:
        valores_preenchidos = df[col].notna().sum()
        percentual = (valores_preenchidos / len(df)) * 100
        status = "✅" if percentual > 50 else "⚠️" if percentual > 10 else "❌"
        print(f"{status} {col}: {valores_preenchidos:,} valores ({percentual:.1f}%)")

# Mostrar amostra dos dados
print(f"\n? AMOSTRA DOS DADOS (primeiras 3 linhas):")
display(df[colunas_gaps].head(3))

In [ ]:
# Função para combinar data e hora em datetime
def combinar_data_hora(df, col_data, col_hora):
    """
    Combina colunas de data e hora em um datetime único
    """
    resultado = []
    
    for i in range(len(df)):
        try:
            data = df.iloc[i][col_data]
            hora = df.iloc[i][col_hora]
            
            # Verificar se ambos existem
            if pd.notna(data) and pd.notna(hora):
                # Converter data para datetime se necessário
                if isinstance(data, str):
                    data = pd.to_datetime(data, errors='coerce')
                
                # Combinar data e hora
                if isinstance(hora, str):
                    # Tentar converter string de hora
                    try:
                        hora_dt = pd.to_datetime(hora, format='%H:%M', errors='coerce').time()
                        datetime_completo = pd.Timestamp.combine(data.date(), hora_dt)
                    except:
                        datetime_completo = pd.NaT
                else:
                    datetime_completo = data
                
                resultado.append(datetime_completo)
            else:
                resultado.append(pd.NaT)
                
        except Exception as e:
            resultado.append(pd.NaT)
    
    return pd.Series(resultado)

print("✅ Função de combinação data/hora criada!")

In [ ]:
# Identificar e combinar colunas de data/hora
colunas = df.columns.tolist()
print("🔍 MAPEAMENTO DAS COLUNAS DE TEMPO:")

# Mapear colunas esperadas
mapa_colunas = {
    'data_carregamento': None,
    'data_retorno_simbolico': None,
    'hora_retorno_simbolico': None,
    'hora_recebimento_nf': None,
    'data_ticket': None,
    'hora_ticket': None,
    'hora_senha': None,
    'hora_gate': None,
    'data_liberacao': None,
    'hora_liberacao': None
}

# Identificar colunas automaticamente
for col in colunas:
    col_lower = col.lower()
    
    if 'data' in col_lower and 'carregamento' in col_lower:
        mapa_colunas['data_carregamento'] = col
    elif 'data' in col_lower and 'retorno' in col_lower:
        mapa_colunas['data_retorno_simbolico'] = col
    elif 'hora' in col_lower and 'retorno' in col_lower:
        mapa_colunas['hora_retorno_simbolico'] = col
    elif 'hora' in col_lower and 'recebimento' in col_lower:
        mapa_colunas['hora_recebimento_nf'] = col
    elif 'data' in col_lower and 'ticket' in col_lower:
        mapa_colunas['data_ticket'] = col
    elif 'hora' in col_lower and 'ticket' in col_lower:
        mapa_colunas['hora_ticket'] = col
    elif 'hora' in col_lower and 'senha' in col_lower:
        mapa_colunas['hora_senha'] = col
    elif 'hora' in col_lower and 'gate' in col_lower:
        mapa_colunas['hora_gate'] = col
    elif 'data' in col_lower and 'liberação' in col_lower:
        mapa_colunas['data_liberacao'] = col
    elif 'hora' in col_lower and 'liberação' in col_lower:
        mapa_colunas['hora_liberacao'] = col

# Exibir mapeamento
for chave, valor in mapa_colunas.items():
    status = "✅" if valor else "❌"
    print(f"{status} {chave}: {valor}")

In [ ]:
# Criar timestamps combinados para análise
print("🔄 CRIANDO TIMESTAMPS COMBINADOS...")

# Cópia do dataframe para trabalhar
df_analise = df.copy()

# Criar timestamps combinados onde possível
if mapa_colunas['data_retorno_simbolico'] and mapa_colunas['hora_retorno_simbolico']:
    df_analise['timestamp_retorno_simbolico'] = combinar_data_hora(
        df_analise, 
        mapa_colunas['data_retorno_simbolico'], 
        mapa_colunas['hora_retorno_simbolico']
    )
    print("✅ Timestamp retorno simbólico criado")

if mapa_colunas['hora_recebimento_nf']:
    # Assumindo que usa a mesma data do retorno simbólico se não houver data específica
    df_analise['timestamp_recebimento_nf'] = pd.to_datetime(
        df_analise[mapa_colunas['hora_recebimento_nf']], errors='coerce'
    )
    print("✅ Timestamp recebimento NF criado")

if mapa_colunas['data_ticket'] and mapa_colunas['hora_ticket']:
    df_analise['timestamp_ticket'] = combinar_data_hora(
        df_analise, 
        mapa_colunas['data_ticket'], 
        mapa_colunas['hora_ticket']
    )
    print("✅ Timestamp ticket criado")

if mapa_colunas['hora_gate']:
    df_analise['timestamp_gate'] = pd.to_datetime(
        df_analise[mapa_colunas['hora_gate']], errors='coerce'
    )
    print("✅ Timestamp gate criado")

if mapa_colunas['data_liberacao'] and mapa_colunas['hora_liberacao']:
    df_analise['timestamp_liberacao'] = combinar_data_hora(
        df_analise, 
        mapa_colunas['data_liberacao'], 
        mapa_colunas['hora_liberacao']
    )
    print("✅ Timestamp liberação criado")

print(f"\n📊 DataFrame de análise criado com {len(df_analise)} registros")

In [ ]:
# ANÁLISE 1: CALCULAR GAPS TEMPORAIS
print("⏱️ CALCULANDO GAPS TEMPORAIS...")

# Gap Cliente: Retorno Simbólico → Recebimento NF Venda
if 'timestamp_retorno_simbolico' in df_analise.columns and 'timestamp_recebimento_nf' in df_analise.columns:
    df_analise['gap_cliente_horas'] = (
        df_analise['timestamp_recebimento_nf'] - df_analise['timestamp_retorno_simbolico']
    ).dt.total_seconds() / 3600
    print("✅ Gap Cliente calculado (horas)")

# Gap Pátio: Ticket → Liberação  
if 'timestamp_ticket' in df_analise.columns and 'timestamp_liberacao' in df_analise.columns:
    df_analise['gap_patio_horas'] = (
        df_analise['timestamp_liberacao'] - df_analise['timestamp_ticket']
    ).dt.total_seconds() / 3600
    print("✅ Gap Pátio calculado (horas)")

# Gap Gate: Ticket → Gate
if 'timestamp_ticket' in df_analise.columns and 'timestamp_gate' in df_analise.columns:
    df_analise['gap_ticket_gate_horas'] = (
        df_analise['timestamp_gate'] - df_analise['timestamp_ticket']
    ).dt.total_seconds() / 3600
    print("✅ Gap Ticket→Gate calculado (horas)")

# Gap Liberação: Gate → Liberação
if 'timestamp_gate' in df_analise.columns and 'timestamp_liberacao' in df_analise.columns:
    df_analise['gap_gate_liberacao_horas'] = (
        df_analise['timestamp_liberacao'] - df_analise['timestamp_gate']
    ).dt.total_seconds() / 3600
    print("✅ Gap Gate→Liberação calculado (horas)")

print("\n📈 GAPS CALCULADOS COM SUCESSO!")

In [ ]:
# ANÁLISE 2: IDENTIFICAR PROCESSOS INCOMPLETOS
print("🔍 IDENTIFICANDO PROCESSOS INCOMPLETOS...")

# Definir campos críticos para processo completo
campos_criticos = []
if 'timestamp_retorno_simbolico' in df_analise.columns:
    campos_criticos.append('timestamp_retorno_simbolico')
if 'timestamp_recebimento_nf' in df_analise.columns:
    campos_criticos.append('timestamp_recebimento_nf')
if 'timestamp_ticket' in df_analise.columns:
    campos_criticos.append('timestamp_ticket')
if 'timestamp_liberacao' in df_analise.columns:
    campos_criticos.append('timestamp_liberacao')

# Identificar processos incompletos
df_analise['processo_completo'] = True
df_analise['campos_faltantes'] = ''

for i, row in df_analise.iterrows():
    faltantes = []
    for campo in campos_criticos:
        if pd.isna(row[campo]):
            faltantes.append(campo.replace('timestamp_', ''))
    
    if faltantes:
        df_analise.loc[i, 'processo_completo'] = False
        df_analise.loc[i, 'campos_faltantes'] = ', '.join(faltantes)

# Estatísticas de completude
total_processos = len(df_analise)
processos_completos = df_analise['processo_completo'].sum()
processos_incompletos = total_processos - processos_completos

print(f"📊 ESTATÍSTICAS DE COMPLETUDE:")
print(f"✅ Processos Completos: {processos_completos} ({processos_completos/total_processos*100:.1f}%)")
print(f"❌ Processos Incompletos: {processos_incompletos} ({processos_incompletos/total_processos*100:.1f}%)")
print(f"📋 Total: {total_processos}")

if processos_incompletos > 0:
    print(f"\n🔍 CAMPOS MAIS FALTANTES:")
    campos_faltantes = df_analise[df_analise['processo_completo'] == False]['campos_faltantes'].str.split(', ').explode()
    contagem_faltantes = campos_faltantes.value_counts()
    print(contagem_faltantes)

In [ ]:
# ANÁLISE 3: ESTATÍSTICAS DOS GAPS
print("📈 ESTATÍSTICAS DOS GAPS TEMPORAIS:")

# Filtrar apenas processos com gaps válidos
df_completos = df_analise[df_analise['processo_completo'] == True]

gaps_para_analisar = []
if 'gap_cliente_horas' in df_analise.columns:
    gaps_para_analisar.append(('Gap Cliente (Retorno→NF)', 'gap_cliente_horas'))
if 'gap_patio_horas' in df_analise.columns:
    gaps_para_analisar.append(('Gap Pátio (Ticket→Liberação)', 'gap_patio_horas'))
if 'gap_ticket_gate_horas' in df_analise.columns:
    gaps_para_analisar.append(('Gap Ticket→Gate', 'gap_ticket_gate_horas'))
if 'gap_gate_liberacao_horas' in df_analise.columns:
    gaps_para_analisar.append(('Gap Gate→Liberação', 'gap_gate_liberacao_horas'))

for nome_gap, coluna_gap in gaps_para_analisar:
    dados_validos = df_completos[coluna_gap].dropna()
    if len(dados_validos) > 0:
        print(f"\n⏱️ {nome_gap}:")
        print(f"   📊 Média: {dados_validos.mean():.1f} horas")
        print(f"   📊 Mediana: {dados_validos.median():.1f} horas")
        print(f"   📊 Mínimo: {dados_validos.min():.1f} horas")
        print(f"   📊 Máximo: {dados_validos.max():.1f} horas")
        print(f"   📊 Desvio Padrão: {dados_validos.std():.1f} horas")
        print(f"   📊 Casos válidos: {len(dados_validos)}")
        
        # Identificar casos extremos (gaps > 24h)
        extremos = dados_validos[dados_validos > 24]
        if len(extremos) > 0:
            print(f"   ⚠️ Casos > 24h: {len(extremos)} ({len(extremos)/len(dados_validos)*100:.1f}%)")

In [ ]:
# DASHBOARD 1: VISÃO GERAL DE PROCESSOS
print("📊 CRIANDO DASHBOARD DE VISÃO GERAL...")

# Gráfico de pizza - Processos Completos vs Incompletos
fig_completude = go.Figure(data=[go.Pie(
    labels=['Processos Completos', 'Processos Incompletos'],
    values=[processos_completos, processos_incompletos],
    hole=.3,
    marker_colors=['#2E8B57', '#DC143C']
)])

fig_completude.update_layout(
    title={
        'text': f'📋 Completude dos Processos (Total: {total_processos})',
        'x': 0.5,
        'font': {'size': 16}
    },
    showlegend=True,
    height=400
)

fig_completude.show()

# Métricas principais
print(f"\n🎯 MÉTRICAS PRINCIPAIS:")
print(f"📋 Total de Processos: {total_processos}")
print(f"✅ Taxa de Completude: {processos_completos/total_processos*100:.1f}%")
print(f"❌ Processos Incompletos: {processos_incompletos}")

In [ ]:
# DASHBOARD 2: ANÁLISE DE GAPS TEMPORAIS
print("📈 CRIANDO DASHBOARD DE GAPS TEMPORAIS...")

if gaps_para_analisar:
    # Criar subplots para histogramas dos gaps
    n_gaps = len(gaps_para_analisar)
    fig_gaps = make_subplots(
        rows=2, cols=2,
        subplot_titles=[nome for nome, _ in gaps_para_analisar[:4]],
        vertical_spacing=0.12
    )
    
    posicoes = [(1,1), (1,2), (2,1), (2,2)]
    
    for i, (nome_gap, coluna_gap) in enumerate(gaps_para_analisar[:4]):
        dados_validos = df_completos[coluna_gap].dropna()
        if len(dados_validos) > 0:
            # Filtrar outliers extremos para melhor visualização
            q95 = dados_validos.quantile(0.95)
            dados_filtrados = dados_validos[dados_validos <= q95]
            
            fig_gaps.add_trace(
                go.Histogram(
                    x=dados_filtrados,
                    nbinsx=20,
                    name=nome_gap,
                    marker_color='#1f77b4',
                    opacity=0.7
                ),
                row=posicoes[i][0], col=posicoes[i][1]
            )
            
            # Adicionar linha da média
            media = dados_validos.mean()
            fig_gaps.add_vline(
                x=media, line_dash="dash", line_color="red",
                annotation_text=f"Média: {media:.1f}h",
                row=posicoes[i][0], col=posicoes[i][1]
            )
    
    fig_gaps.update_layout(
        title_text="⏱️ Distribuição dos Gaps Temporais (horas)",
        showlegend=False,
        height=600
    )
    
    fig_gaps.update_xaxes(title_text="Horas")
    fig_gaps.update_yaxes(title_text="Frequência")
    
    fig_gaps.show()

else:
    print("❌ Não foi possível criar gráficos de gaps - dados insuficientes")

In [ ]:
# DASHBOARD 3: LINHA DO TEMPO DE PROCESSOS SELECIONADOS
print("🕐 CRIANDO LINHA DO TEMPO DOS PROCESSOS...")

# Selecionar processos completos para linha do tempo
df_timeline = df_completos.head(10)  # Primeiros 10 processos completos

if len(df_timeline) > 0:
    fig_timeline = go.Figure()
    
    # Definir eventos e cores
    eventos = []
    cores = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57']
    
    if 'timestamp_retorno_simbolico' in df_timeline.columns:
        eventos.append(('Retorno Simbólico', 'timestamp_retorno_simbolico', cores[0]))
    if 'timestamp_recebimento_nf' in df_timeline.columns:
        eventos.append(('Recebimento NF', 'timestamp_recebimento_nf', cores[1]))
    if 'timestamp_ticket' in df_timeline.columns:
        eventos.append(('Ticket', 'timestamp_ticket', cores[2]))
    if 'timestamp_gate' in df_timeline.columns:
        eventos.append(('Gate', 'timestamp_gate', cores[3]))
    if 'timestamp_liberacao' in df_timeline.columns:
        eventos.append(('Liberação', 'timestamp_liberacao', cores[4]))
    
    # Adicionar pontos para cada evento
    for nome_evento, coluna_evento, cor in eventos:
        dados_validos = df_timeline[coluna_evento].dropna()
        indices = dados_validos.index
        
        fig_timeline.add_trace(go.Scatter(
            x=dados_validos,
            y=[f"Processo {i}" for i in indices],
            mode='markers',
            name=nome_evento,
            marker=dict(size=10, color=cor),
            text=[f"{nome_evento}<br>Processo {i}" for i in indices],
            hovertemplate='%{text}<br>%{x}<extra></extra>'
        ))
    
    fig_timeline.update_layout(
        title="🕐 Linha do Tempo dos Processos (Primeiros 10 Completos)",
        xaxis_title="Data/Hora",
        yaxis_title="Processos",
        height=600,
        showlegend=True
    )
    
    fig_timeline.show()

else:
    print("❌ Não há processos completos suficientes para criar linha do tempo")

In [ ]:
# RELATÓRIO EXECUTIVO PARA O GERENTE
print("📋 RELATÓRIO EXECUTIVO - GAPS TERLOC")
print("=" * 50)

print(f"\n🎯 RESUMO GERAL:")
print(f"   📊 Total de Processos Analisados: {total_processos}")
print(f"   ✅ Processos Completos: {processos_completos} ({processos_completos/total_processos*100:.1f}%)")
print(f"   ❌ Processos Incompletos: {processos_incompletos} ({processos_incompletos/total_processos*100:.1f}%)")

if gaps_para_analisar:
    print(f"\n⏱️ GAPS TEMPORAIS PRINCIPAIS:")
    for nome_gap, coluna_gap in gaps_para_analisar:
        dados_validos = df_completos[coluna_gap].dropna()
        if len(dados_validos) > 0:
            media = dados_validos.mean()
            casos_24h = len(dados_validos[dados_validos > 24])
            print(f"   📈 {nome_gap}:")
            print(f"      • Tempo Médio: {media:.1f} horas")
            print(f"      • Casos > 24h: {casos_24h} ({casos_24h/len(dados_validos)*100:.1f}%)")

print(f"\n🚨 ALERTAS E RECOMENDAÇÕES:")
if processos_incompletos > total_processos * 0.3:
    print(f"   ⚠️ ALTA taxa de processos incompletos ({processos_incompletos/total_processos*100:.1f}%)")
    print(f"   💡 Revisar processos de coleta de dados e follow-up")

if 'gap_cliente_horas' in df_analise.columns:
    gap_cliente_dados = df_completos['gap_cliente_horas'].dropna()
    if len(gap_cliente_dados) > 0 and gap_cliente_dados.mean() > 12:
        print(f"   ⚠️ Gap Cliente muito alto (média: {gap_cliente_dados.mean():.1f}h)")
        print(f"   💡 Melhorar comunicação com clientes sobre prazos de NF")

if 'gap_patio_horas' in df_analise.columns:
    gap_patio_dados = df_completos['gap_patio_horas'].dropna()
    if len(gap_patio_dados) > 0 and gap_patio_dados.mean() > 8:
        print(f"   ⚠️ Gap Pátio alto (média: {gap_patio_dados.mean():.1f}h)")
        print(f"   💡 Otimizar processo interno de liberação")

print(f"\n✅ PRÓXIMOS PASSOS SUGERIDOS:")
print(f"   1. Implementar follow-up automático para processos incompletos")
print(f"   2. Definir SLAs para cada etapa do processo")
print(f"   3. Criar alertas para gaps > 24h")
print(f"   4. Analisar causas dos gaps mais longos")
print(f"   5. Implementar dashboards em tempo real")

In [ ]:
# SALVAR DADOS PROCESSADOS
print("💾 SALVANDO DADOS PROCESSADOS...")

# Salvar dataframe com análises
arquivo_saida = 'dados_gaps_terloc_processados.csv'
df_analise.to_csv(arquivo_saida, index=False, encoding='utf-8-sig')
print(f"✅ Dados salvos em: {arquivo_saida}")

# Resumo das colunas criadas
print(f"\n📋 COLUNAS ADICIONADAS NA ANÁLISE:")
colunas_novas = [
    'timestamp_retorno_simbolico', 'timestamp_recebimento_nf', 
    'timestamp_ticket', 'timestamp_gate', 'timestamp_liberacao',
    'gap_cliente_horas', 'gap_patio_horas', 'gap_ticket_gate_horas', 
    'gap_gate_liberacao_horas', 'processo_completo', 'campos_faltantes'
]

for col in colunas_novas:
    if col in df_analise.columns:
        print(f"   ✅ {col}")
    else:
        print(f"   ❌ {col} (não criada)")

print(f"\n🎯 ANÁLISE CONCLUÍDA! Use este notebook para monitoramento contínuo dos gaps TERLOC.")